In [1]:
import pandas as pd

In [5]:
import requests
import time
import random
import csv
import datetime


# --- CẤU HÌNH NGƯỜI DÙNG (BẠN CHỈ CẦN SỬA DÒNG NÀY) ---
# Copy toàn bộ nội dung 'cookie' trong tab Headers dán vào giữa cặp nháy đơn ''
my_cookie = 'ZConsent=timestamp=1760327803197&location=https://zalo.me/pc; __zi=3000.SSZzejyD0jydXQckra00a3BBfxQL71AQV8UbljfP5v1yYAVmsLWKt7wDflZQNHlTCm.1; __zi-legacy=3000.SSZzejyD0jydXQckra00a3BBfxQL71AQV8UbljfP5v1yYAVmsLWKt7wDflZQNHlTCm.1; ozi=2000.SSZzejyD0jydXQckra00a3BBfxQK71AQVOUaljvP5vPwYQkusn8Vatd5gRtQ6XoME3ap.1; ZConsent=timestamp=1760327803197&location=https://zalo.me/pc; _fbp=fb.1.1763434116272.510980907223108041; _ga_RYD7END4JE=GS2.2.s1763434193$o2$g1$t1763434194$j59$l0$h0; _ga_YS1V643LGV=GS2.1.s1763434192$o2$g0$t1763434493$j60$l0$h0; zpsid=Pod3.282972265.65.rxTiwz9KZnwxX58EtbIF7gSd_IpgPhOfu6QvAxKHm9FCIIYvqzTk1RjKZnu; _gcl_au=1.1.1485582632.1763434115; _hjSessionUser_3705269=eyJpZCI6IjA1YTIzZmZjLTJlY2QtNWU4My1hNTM1LTlhNTE1MWJjY2EyMCIsImNyZWF0ZWQiOjE3NjM0MzQ4ODc2MjEsImV4aXN0aW5nIjp0cnVlfQ==; _ga_S4Y4BPVHZV=GS2.1.s1763434942$o1$g1$t1763434943$j59$l0$h0; _ga_C51H66N22P=GS2.2.s1763448660$o2$g0$t1763448660$j60$l0$h0; _ga_4QSSY47VGF=GS2.2.s1763448660$o2$g0$t1763448660$j60$l0$h0; _ga_GP4XVYJ45L=GS2.1.s1763448660$o2$g0$t1763448660$j60$l0$h0; _zlang=vn; _gid=GA1.2.613380716.1764146568; _ga_3EM8ZPYYN3=GS2.2.s1764146568$o11$g0$t1764146568$j60$l0$h0; ZPIDaefdd=s392mzlcc8tu11cjvukyn7sp7; ZPIDff384=6skq4e7rhjb11ol182v2yvs6a; __oam_sid=58b9814978b20bed52a2.1+JHo8J4vqQLsqblaNszktesJRawodOe1DE7+FjhQFQ=; zlanguage=VN; __zactoa_fb8cc3a622f3cbad92e2=MTkzNzAzNTExNTU1NjM0NTg4OQ==; __zapage=1a10aeb2ebf802a65be9; _ga_WSPJQT0ZH1=GS2.1.s1764147323$o2$g0$t1764147334$j49$l0$h0; zoalang=vn; _ga=GA1.2.264635782.1760327782; _ga_NVN38N77J3=GS2.2.s1764147308$o3$g1$t1764147339$j29$l0$h0; _ga_E63JS7SPBL=GS2.1.s1764147308$o3$g1$t1764147340$j28$l0$h0; zoats=afc56d11adeadeb587fa'


def get_broadcast_history_and_save():
    url = "https://oa.zalo.me/manage/broadcast/history"
   
    # Headers chuẩn theo trình duyệt Chrome 142 của bạn
    headers = {
        'authority': 'oa.zalo.me',
        'accept': '*/*',
        'accept-language': 'vi-VN,vi;q=0.9,fr-FR;q=0.8,fr;q=0.7,en-US;q=0.6,en;q=0.5',
        'cookie': my_cookie,
        'referer': 'https://oa.zalo.me/manage/broadcast/compose',
        'sec-ch-ua': '"Chromium";v="142", "Google Chrome";v="142", "Not_A Brand";v="99"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-origin',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36',
    }


    all_data = []
    page = 1
   
    print(">>> Bắt đầu lấy dữ liệu...")


    while True:
        print(f"--- Đang tải trang {page} ---")
       
        params = {
            'action': 'getdata',
            'pageIndex': str(page),
            'pageCount': '10', # Giữ nguyên 10 để an toàn
            'fromDate': '',
            'toDate': ''
        }


        try:
            response = requests.get(url, headers=headers, params=params)
           
            # 1. Kiểm tra lỗi Cookie hết hạn
            if "login" in response.url or "<!DOCTYPE html>" in response.text:
                print("❌ LỖI: Cookie đã hết hạn hoặc không đúng!")
                print("👉 Vui lòng F12 lấy lại cookie mới và dán lại vào code.")
                break
           
            # 2. Xử lý dữ liệu JSON
            data = response.json()
            items = data.get('broadcasts', [])
           
            if not items:
                print("✅ Đã hết dữ liệu để lấy.")
                break
               
            # 3. Lọc lấy các trường cần thiết
            for item in items:
                articles = item.get('articles', [])

                title = None
                if isinstance(articles, list) and len(articles) > 0:
                    title = articles[0].get('title')

                row = {
                    'ID': item.get('broadcastId'),
                    'Tiêu đề' : title,
                    'Trạng thái': item.get('statusInfo'), # Thường là 1, 2, 3 (cần map nếu muốn hiện chữ)
                    'Lượt xem': item.get('numberReceive', 0),
                    'Lượt gửi': item.get('numberSend', 0),
                    'Lượt click': item.get('numberView', 0), # Nếu có
                    'Ngày tạo': item.get('time')
                }
                all_data.append(row)
                print(f"   + Đã lấy: {row['Tiêu đề']}")
           
            # 4. Ngủ ngẫu nhiên để tránh bị chặn (QUAN TRỌNG)
            sleep_time = random.uniform(2, 5)
            time.sleep(sleep_time)
           
            page += 1
           
        except Exception as e:
            print(f"❌ Có lỗi xảy ra: {e}")
            break


    # --- LƯU RA FILE CSV (Mở được bằng Excel) ---
    if all_data:
        filename = f"zalo_broadcast_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
       
        # 'utf-8-sig' giúp Excel hiển thị đúng tiếng Việt
        with open(filename, 'w', newline='', encoding='utf-8-sig') as f:
            writer = csv.DictWriter(f, fieldnames=all_data[0].keys())
            writer.writeheader()
            writer.writerows(all_data)
           
        print(f"\n🎉 HOÀN TẤT! Đã lưu {len(all_data)} dòng vào file: {filename}")
    else:
        print("\n⚠️ Không lấy được dữ liệu nào.")


if __name__ == "__main__":
    if my_cookie == 'DÁN_COOKIE_CỦA_BẠN_VÀO_ĐÂY':
        print("❌ BẠN CHƯA DÁN COOKIE!")
        print("👉 Hãy sửa dòng số 8 trong code: my_cookie = 'copy_cookie_paste_vao_day'")
    else:
        get_broadcast_history_and_save()


>>> Bắt đầu lấy dữ liệu...
--- Đang tải trang 1 ---
   + Đã lấy: Hhhhhhhhhhhhhhhhhhh
   + Đã lấy: BẬT VIBE CHILL – TRI ÂN “THẦY IU” MỪNG 20/11
   + Đã lấy: HÒA VÀO ĐÊM HALLOWEEN BUNG NÓC
   + Đã lấy: KHUẤY ĐẢO CÙNG HỘI CHỊ EM VỚI HER-ICANE NIGHT
   + Đã lấy: DZÔKTOBERFEST - Enjoy Oktoberfest kiểu Việt
   + Đã lấy: VẼ ĐÈN XINH XINH – TRUNG THU BLINK BLINK
   + Đã lấy: CUỐI TUẦN XẢ STRESS CÙNG SIPPIN’ SUNDAY
   + Đã lấy: CHAY O’CLOCK – NHẬU CHAY VẪN CHÁYYY 🌱🍻
   + Đã lấy: TỰ HÀO HƯỞNG VỊ TỰ DO – MỪNG ĐẠI LỄ 2/9 TẠI BA GÁC
   + Đã lấy: Không thể bỏ lỡ SUMMÊ-LY SPRITZ
--- Đang tải trang 2 ---
   + Đã lấy: 🍻 Khám phá hương vị Bỉ cùng Belgo Tap Takeover
   + Đã lấy: 🍻Thưởng thức vị mùa hè mê ly cùng TAP TAKEOVER
   + Đã lấy: HỘI CẠ CỨNG ĐỪNG BỎ LỠ “BESTIE NIGHT” TỐI THỨ BẢYYY
   + Đã lấy: ĐỪNG BỎ LỠ LADIES NIGHT TỐI THỨ SÁUUU
   + Đã lấy: CHẤN ĐỘNG VỚI DEAL $1 MỖI NGÀY TẠI TẤT CẢ CHI NHÁNH!!!
   + Đã lấy: 🧡CHÚC MỪNG NGÀY GIA ĐÌNH VIỆT NAM 28/06
   + Đã lấy: ✈️ LÊN NÓC CUỐI TUẦN TẠI BA GÁC NA

In [6]:
import requests

url = "https://www.facebook.com/data/manifest/?is_workplace_mobile_pwa_dogfooding=0"

cookies = {
    "c_user": "100014844849106",
    "xs": "32%3AmYDg6BNBRghsVw%3A2%3A1754712253%3A-1%3A-1%3A%3AAcwb8gMWCo5EzKf_kWBKAyBFvjmSuvYNPvADh3_R0pDA",
    "datr": "cciWaDlYfs-BkbyxAKOsnoLA",
    "sb": "cciWaP4G179hzdP_pdHqdI0i",
    "fr": "1knBfRW2XvuS2EKvL.AWcjPpShz6fpjzL4hfQCBvIK4BZxkIuVDFoTlyAW5q3Pgpfn1lk.BpKCmc..AAA.0.0.BpKDU1.AWdrxuXRaP5YfhOQabhR0CcEZNk",
    "presence": "C%7B%22t3%22%3A%5B%5D%2C%22utc3%22%3A1764241901152%2C%22v%22%3A1%7D",
    "wd": "162x695",
}

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                  "(KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36",
    "Referer": "https://www.facebook.com/",
    "Accept": "*/*",
}

response = requests.get(url, cookies=cookies, headers=headers)

print(response.status_code)
print(response.text[:500])  # print first 500 chars


200
{"gcm_sender_id":"15057814354","gcm_user_visible_only":true,"edge_side_panel":{"preferred_width":376},"short_name":"Facebook","name":"Facebook","start_url":"\/?ref=homescreenpwa","display":"minimal-ui","background_color":"#FFFFFF","theme_color":"#1877F2","icons":[{"src":"https:\/\/static.xx.fbcdn.net\/rsrc.php\/v4\/y0\/r\/eFZD1KABzRA.png","sizes":"192x192","type":"image\/png"},{"src":"https:\/\/static.xx.fbcdn.net\/rsrc.php\/v4\/yd\/r\/DeNyZD1Vj3q.png","sizes":"512x512","type":"image\/png"}],"la


In [16]:
import requests
import json

# 1. Cấu hình thông tin API
api_url = "https://0314142245.minvoice.site/api/Account/Login"

# 2. Cấu hình Headers
headers = {
    "Content-Type": "application/json"
}

# 3. Dữ liệu Input (Bạn nhớ thay đổi username và password thực tế)
payload = {
    "username": "0314142245",       # Thay bằng tài khoản thật của bạn
    "password": "B@Gac1812pacc"   # Thay bằng mật khẩu thật của bạn
}

try:
    # 4. Thực hiện gửi request POST
    response = requests.post(api_url, json=payload, headers=headers)

    # 5. Kiểm tra kết quả
    if response.status_code == 200:
        print("Đăng nhập thành công!")
        data = response.json()
        
        # In toàn bộ phản hồi để xem cấu trúc
        print("Response:", json.dumps(data, indent=4))
        
        # Ví dụ cách lấy token ra (giả sử key trả về là 'token' hoặc 'access_token')
        # token = data.get("token") 
        # print("Token của bạn là:", token)
    else:
        print(f"Đăng nhập thất bại. Mã lỗi: {response.status_code}")
        print("Nội dung lỗi:", response.text)

except Exception as e:
    print(f"Có lỗi xảy ra: {e}")

Đăng nhập thành công!
Response: {
    "code": "99",
    "message": "Request to external service failed.. TraceId: 169e03d48565061f47f0ab40f29d8858",
    "ok": false,
    "error": "Request to external service failed.. TraceId: 169e03d48565061f47f0ab40f29d8858"
}


In [22]:
import requests
import json

# 1. Cấu hình thông tin API
api_url = "https://0106026495-999.minvoice.site/api/Account/Login"

# 2. Cấu hình Headers
headers = {
    "Content-Type": "application/json"
}

# 3. Dữ liệu Input
payload = {
    "username": "admin",            
    "password": "m3!siUK!w@i@C3",   
    "ma_dvcs": "0314142245"  # <--- BẮT BUỘC PHẢI CÓ DÒNG NÀY (Theo ảnh bạn gửi là "VP")
}

try:
    # 4. Thực hiện gửi request POST
    response = requests.post(api_url, json=payload, headers=headers)

    # 5. Kiểm tra kết quả
    if response.status_code == 200:
        print("Đăng nhập thành công!")
        data = response.json()
        print("Response:", json.dumps(data, indent=4))
    else:
        print(f"Đăng nhập thất bại. Mã lỗi: {response.status_code}")
        print("Nội dung lỗi:", response.text)

except Exception as e:
    print(f"Có lỗi xảy ra: {e}")

Đăng nhập thành công!
Response: {
    "code": "99",
    "message": "T\u00ean \u0111\u0103ng nh\u1eadp ho\u1eb7c m\u1eadt kh\u1ea9u kh\u00f4ng \u0111\u00fang. TraceId: adbd16a7cd6d7eb30147b96991d8ee9c",
    "ok": false,
    "error": "T\u00ean \u0111\u0103ng nh\u1eadp ho\u1eb7c m\u1eadt kh\u1ea9u kh\u00f4ng \u0111\u00fang. TraceId: adbd16a7cd6d7eb30147b96991d8ee9c"
}


In [ ]:
{
    "code": "00",
    "message": "Thành công",
    "ok": true,
    "token": "O87316arj5+Od3Fqyy5hzdBfIuPk73eKqpAzBSvv8sY="
}

In [29]:
import requests
import json

# ==============================================================================
# 1. NHẬP TOKEN CỦA BẠN VÀO ĐÂY
# ==============================================================================
your_token = "O87316arj5+Od3Fqyy5hzdBfIuPk73eKqpAzBSvv8sY="  # <-- Dán token vào giữa dấu nháy kép

# 2. Cấu hình API
api_url = "https://0314142245.minvoice.site/api/InvoiceApi78/GetInvoices"

# 3. Cấu hình Headers (Quan trọng: Phải có Authorization)
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {your_token}" 
    # Lưu ý: Thông thường chuẩn là "Bearer <token>". 
    # Trong ảnh ghi là "Bear Token", nếu code này lỗi 401, bạn thử sửa chữ "Bearer" thành "Bear" nhé.
}

# 4. Dữ liệu Input (Payload) theo ảnh bạn gửi
payload = {
    "tuNgay": "2024-01-01",
    "denngay": "2025-12-22",
    "khieu": "1C25MPA",
    "start": 0,
    "count": 20,
    "coChiTiet": True  # Trong Python, true viết là True (viết hoa chữ cái đầu)
}

try:
    # 5. Gửi request POST
    response = requests.post(api_url, json=payload, headers=headers)

    # 6. Kiểm tra kết quả
    if response.status_code == 200:
        print("Lấy dữ liệu hóa đơn thành công!")
        data = response.json()
        print(json.dumps(data, indent=4, ensure_ascii=False)) # ensure_ascii=False để hiện tiếng Việt
    else:
        print(f"Lỗi khi gọi API. Mã lỗi: {response.status_code}")
        print("Chi tiết:", response.text)

except Exception as e:
    print(f"Có lỗi xảy ra: {e}")

Lấy dữ liệu hóa đơn thành công!
{
    "code": "99",
    "message": "Request to external service failed: GetInvoice. Status Code: InternalServerError, Details: {\"error\":{\"code\":null,\"message\":\"An exception has been raised that is likely due to a transient failure.\",\"details\":\"InvalidOperationException: An exception has been raised that is likely due to a transient failure.\\r\\nSTACK TRACE:    at Npgsql.EntityFrameworkCore.PostgreSQL.Storage.Internal.NpgsqlExecutionStrategy.Execute[TState,TResult](TState state, Func`3 operation, Func`3 verifySucceeded)\\r\\n   at Microsoft.EntityFrameworkCore.Query.Internal.SingleQueryingEnumerable`1.Enumerator.MoveNext()\\r\\n   at System.Collections.Generic.List`1..ctor(IEnumerable`1 collection)\\r\\n   at System.Linq.Enumerable.ToList[TSource](IEnumerable`1 source)\\r\\n   at MInvoice.Invoices.MasterDataService.GetTenantSettingAsync() in D:\\\\Work\\\\Minvoice\\\\InvoiceApp\\\\Source\\\\einvoice2.0\\\\src\\\\MInvoice.Application\\\\MasterD

In [ ]:
import requests
import json

# ==============================================================================
# 1. TOKEN
# ==============================================================================
your_token = "O87316arj5+Od3Fqyy5hzdBfIuPk73eKqpAzBSvv8sY=" # <-- Thay token thật vào đây

# ==============================================================================
# 2. CẤU HÌNH API (SỬA LẠI DOMAIN THÀNH .APP CHO GIỐNG POSTMAN)
# ==============================================================================
# Cũ: .site (Sai) -> Mới: .app (Đúng theo ảnh Postman)
api_url = "https://0314142245.minvoice.app/api/InvoiceApi78/GetInvoices"

# ==============================================================================
# 3. HEADERS
# ==============================================================================
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {your_token}",
    "User-Agent": "PostmanRuntime/7.36.0" # Giả lập là Postman để tránh bị chặn
}

# 4. Dữ liệu Input
payload = {
    "tuNgay": "2024-01-01",
    "denngay": "2025-12-22",
    "khieu": "1C25MPA",
    "start": 0,
    "count": 20,
    "coChiTiet": True
}

try:
    print(f"Đang gọi đến: {api_url}")
    response = requests.post(api_url, json=payload, headers=headers, timeout=30)

    # In kết quả
    data = response.json()
    
    # Kiểm tra logic nghiệp vụ (API này trả về 200 nhưng vẫn có thể là lỗi)
    if response.status_code == 200 and data.get("ok") == True:
        print("Lấy dữ liệu THÀNH CÔNG!")
        print(json.dumps(data, indent=4, ensure_ascii=False))
    else:
        print("Gọi API thất bại (Dù Status code có thể là 200)!")
        print(f"Message lỗi: {data.get('message')}")
        print(f"Chi tiết lỗi server: {data.get('error')}")

except Exception as e:
    print(f"Có lỗi xảy ra: {e}")

Đang gọi đến: https://0314142245.minvoice.app/api/InvoiceApi78/GetInvoices
Lấy dữ liệu THÀNH CÔNG!
{
    "code": "00",
    "message": null,
    "ok": true,
    "total": 22557,
    "data": [
        {
            "key_api": "UOnnoxdcsZ",
            "inv_invoiceAuth_id": "3a1e592b-8554-d643-b0d1-131dd2afd76e",
            "inv_originalId": "3a1e592b-8554-d643-b0d1-131dd2afd76e",
            "inv_invoiceIssuedDate": "2025-12-22T00:00:00",
            "inv_invoiceSeries": "1C25MPA",
            "inv_invoiceNumber": 22557,
            "so_benh_an": "00288",
            "inv_currencyCode": "VND",
            "inv_exchangeRate": 1.0,
            "inv_paymentMethodName": "TM/CK",
            "inv_buyerDisplayName": "Người mua không lấy hóa đơn",
            "inv_buyerLegalName": "Người mua không lấy hóa đơn",
            "inv_buyerTaxCode": null,
            "inv_buyerAddressLine": null,
            "inv_buyerEmail": null,
            "inv_buyerBankAccount": null,
            "inv_buyerBankNa

In [1]:
import requests
import json
import pandas as pd
import requests
import json

# ==============================================================================
# 1. TOKEN
# ==============================================================================
your_token = "O87316arj5+Od3Fqyy5hzdBfIuPk73eKqpAzBSvv8sY=" # <-- Thay token thật vào đây

# ==============================================================================
# 2. CẤU HÌNH API (SỬA LẠI DOMAIN THÀNH .APP CHO GIỐNG POSTMAN)
# ==============================================================================
# Cũ: .site (Sai) -> Mới: .app (Đúng theo ảnh Postman)
api_url = "https://0314142245.minvoice.app/api/InvoiceApi78/GetInvoices"

# ==============================================================================
# 3. HEADERS
# ==============================================================================
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {your_token}",
    "User-Agent": "PostmanRuntime/7.36.0" # Giả lập là Postman để tránh bị chặn
}

# 4. Dữ liệu Input
payload = {
    "tuNgay": "2024-01-01",
    "denngay": "2025-12-22",
    "khieu": "1C25MPA",
    "start": 0,
    "count": 20,
    "coChiTiet": True
}

try:
    response = requests.post(api_url, json=payload, headers=headers, timeout=30)
    result = response.json()

    if response.status_code == 200 and result.get("ok"):
        list_data = result.get("data", [])
        df = pd.DataFrame(list_data)
        print(df.head()) 
    else:
        print("Lỗi API:", result.get("message"))

except Exception as e:
    print(f"Lỗi: {e}")

      key_api                    inv_invoiceAuth_id  \
0  UOnnoxdcsZ  3a1e592b-8554-d643-b0d1-131dd2afd76e   
1  SVYZtwwZoz  3a1e592b-5b5a-62fe-b64d-723ba6b61669   
2  JIleYnmobZ  3a1e592b-4405-696c-7468-98df6f73afda   
3  ALmaroib1z  3a1e592b-1aab-0f6a-ede7-7eaa0b0c2288   
4  5NYvkZoc1t  3a1e592a-de61-28c5-a42e-25b0cdf47dce   

                         inv_originalId inv_invoiceIssuedDate  \
0  3a1e592b-8554-d643-b0d1-131dd2afd76e   2025-12-22T00:00:00   
1  3a1e592b-5b5a-62fe-b64d-723ba6b61669   2025-12-22T00:00:00   
2  3a1e592b-4405-696c-7468-98df6f73afda   2025-12-22T00:00:00   
3  3a1e592b-1aab-0f6a-ede7-7eaa0b0c2288   2025-12-22T00:00:00   
4  3a1e592a-de61-28c5-a42e-25b0cdf47dce   2025-12-22T00:00:00   

  inv_invoiceSeries  inv_invoiceNumber so_benh_an inv_currencyCode  \
0           1C25MPA              22557      00288              VND   
1           1C25MPA              22556      00286              VND   
2           1C25MPA              22555      00284              VND  

In [37]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
# pd.set_option('display.max_colwidth', None)
# df.head(5)
df[df['so_benh_an']=='00280'].head(5)

,key_api,inv_invoiceAuth_id,inv_originalId,inv_invoiceIssuedDate,inv_invoiceSeries,inv_invoiceNumber,so_benh_an,inv_currencyCode,inv_exchangeRate,inv_paymentMethodName,inv_buyerDisplayName,inv_buyerLegalName,inv_buyerTaxCode,inv_buyerAddressLine,inv_buyerEmail,inv_buyerBankAccount,inv_buyerBankName,OrtherTax,environmentalProtectionFee,connectorMaintenanceFee,drainageFee,serviceCharge,deductionAmount,ortherFee,deductionVatamount,inv_TotalAmount,inv_discountAmount,inv_vatAmount,inv_TotalAmountWithoutVat,trang_thai,trang_thai_hd,sobaomat,macqt,shdon,ten,tnmua,mst,dchi,buyerTel,buyerIdentityCard,email,hoadon68_id,is_tthdon,tthai,tgtcthue,tgtthue,tgtttbso,isOtherSystem,relatedInvoiceListDate,relatedInvoiceListNumber,relatedInvoiceDate,relatedInvoiceNumber,relatedInvoiceType,relatedInvoiceSerial,relatedTemplateCode,ghi_chu,is_success,note_error,id,phong,doan,ngayden,ngaydi,phidv,nonTaxZone,so_phong,so_folio,inv_arrivalDate,inv_departureDate,tencuahang,dchicuahang,macuahang,ma_dt,ma_voi_bom,bien_so_xe,inv_productCode,inv_productName,socialDeductionAmount,ratioOrtherTax,so_hchieu,mdvqhnsach_nmua,ten_ch,ma_ch,cccd,cccdan,sdtnmua,details
8,0Tayybdnom,3a1e5928-3e22-9e2e-cffe-ee500583123e,3a1e5928-3e22-9e2e-cffe-ee500583123e,2025-12-22T00:00:00,1C25MPA,22549,00280,VND,1.0,TM/CK,LÊ THỊ VÂN TRANG,CÔNG TY TNHH QM PLUS,0317066264,"Số 07 Đường số 12, Phường Long Trường, Thành phố Hồ Chí Minh, Việt Nam",None,None,None,None,None,None,None,None,None,None,None,937440.0,0.0,69440.0,868000.0,4,0,0Tayybdnom,M1-25-DFVCM-00000022560,22549,CÔNG TY TNHH QM PLUS,LÊ THỊ VÂN TRANG,0317066264,"Số 07 Đường số 12, Phường Long Trường, Thành phố Hồ Chí Minh, Việt Nam",None,None,None,3a1e5928-3e22-9e2e-cffe-ee500583123e,0,Đã gửi,868000.0,69440.0,937440.0,None,None,None,None,None,None,None,None,None,1,None,3a1e5928-3e22-9e2e-cffe-ee500583123e,None,None,None,None,0.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"[{'tchat': 1, 'stt_rec0': '1', 'inv_itemCode': 'BG-BO002', 'inv_itemName': 'Khăn Lạnh Cuộn', 'inv_unitCode': 'Cái', 'inv_quantity': 5.0, 'inv_unitPrice': 4000.0, 'phidichvu': None, 'inv_discountPercentage': 0.0, 'inv_discountAmount': 0.0, 'inv_TotalAmountWithoutVat': 20000.0, 'ma_thue': '8', 'inv_vatAmount': 1600.0, 'inv_TotalAmount': 21600.0, 'skhung': None, 'smay': None, 'bien_sxe': None, 'ten_ngui': None, 'mst_ngui': None, 'sddanh_ngui': None, 'dchi_ngui': None}, {'tchat': 1, 'stt_rec0': '2', 'inv_itemCode': 'BG-FSNACK014', 'inv_itemName': 'Snack Bánh Tráng Tôm (Free)', 'inv_unitCode': 'Phần', 'inv_quantity': 2.0, 'inv_unitPrice': 0.0, 'phidichvu': None, 'inv_discountPercentage': 0.0, 'inv_discountAmount': 0.0, 'inv_TotalAmountWithoutVat': 0.0, 'ma_thue': '8', 'inv_vatAmount': 0.0, 'inv_TotalAmount': 0.0, 'skhung': None, 'smay': None, 'bien_sxe': None, 'ten_ngui': None, 'mst_ngui': None, 'sddanh_ngui': None, 'dchi_ngui': None}, {'tchat': 1, 'stt_rec0': '3', 'inv_itemCode': 'BG-FCB2304001', 'inv_itemName': 'Combo BA GÁC', 'inv_unitCode': 'Phần', 'inv_quantity': 1.0, 'inv_unitPrice': 749000.0, 'phidichvu': None, 'inv_discountPercentage': 0.0, 'inv_discountAmount': 0.0, 'inv_TotalAmountWithoutVat': 749000.0, 'ma_thue': '8', 'inv_vatAmount': 59920.0, 'inv_TotalAmount': 808920.0, 'skhung': None, 'smay': None, 'bien_sxe': None, 'ten_ngui': None, 'mst_ngui': None, 'sddanh_ngui': None, 'dchi_ngui': None}, {'tchat': 1, 'stt_rec0': '4', 'inv_itemCode': 'BG-FV2304012', 'inv_itemName': 'Rau Rừng Xào X.O', 'inv_unitCode': 'Phần', 'inv_quantity': 1.0, 'inv_unitPrice': 99000.0, 'phidichvu': None, 'inv_discountPercentage': 0.0, 'inv_discountAmount': 0.0, 'inv_TotalAmountWithoutVat': 99000.0, 'ma_thue': '8', 'inv_vatAmount': 7920.0, 'inv_TotalAmount': 106920.0, 'skhung': None, 'smay': None, 'bien_sxe': None, 'ten_ngui': None, 'mst_ngui': None, 'sddanh_ngui': None, 'dchi_ngui': None}]"


In [13]:
df.columns

Index(['key_api', 'inv_invoiceAuth_id', 'inv_originalId',
       'inv_invoiceIssuedDate', 'inv_invoiceSeries', 'inv_invoiceNumber',
       'so_benh_an', 'inv_currencyCode', 'inv_exchangeRate',
       'inv_paymentMethodName', 'inv_buyerDisplayName', 'inv_buyerLegalName',
       'inv_buyerTaxCode', 'inv_buyerAddressLine', 'inv_buyerEmail',
       'inv_buyerBankAccount', 'inv_buyerBankName', 'OrtherTax',
       'environmentalProtectionFee', 'connectorMaintenanceFee', 'drainageFee',
       'serviceCharge', 'deductionAmount', 'ortherFee', 'deductionVatamount',
       'inv_TotalAmount', 'inv_discountAmount', 'inv_vatAmount',
       'inv_TotalAmountWithoutVat', 'trang_thai', 'trang_thai_hd', 'sobaomat',
       'macqt', 'shdon', 'ten', 'tnmua', 'mst', 'dchi', 'buyerTel',
       'buyerIdentityCard', 'email', 'hoadon68_id', 'is_tthdon', 'tthai',
       'tgtcthue', 'tgtthue', 'tgtttbso', 'isOtherSystem',
       'relatedInvoiceListDate', 'relatedInvoiceListNumber',
       'relatedInvoiceDate', 're

In [10]:
df.to_excel('file_name.xlsx', index=False)

ModuleNotFoundError: No module named 'openpyxl'

In [12]:
import requests
import pandas as pd

# 1. Cấu hình
your_token = "O87316arj5+Od3Fqyy5hzdBfIuPk73eKqpAzBSvv8sY=" # Thay token của bạn
api_url = "https://0314142245.minvoice.app/api/InvoiceApi78/GetInvoices"

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {your_token}",
    "User-Agent": "PostmanRuntime/7.36.0"
}

payload = {
    "tuNgay": "2024-01-01",
    "denngay": "2025-12-22",
    "khieu": "1C25MPA",
    "start": 0,
    "count": 20,
    "coChiTiet": True
}

try:
    response = requests.post(api_url, json=payload, headers=headers, timeout=30)
    result = response.json()

    if response.status_code == 200 and result.get("ok"):
        raw_data = result.get("data", [])

        if raw_data:
            # --- XỬ LÝ DATAFRAME ĐỂ XUẤT EXCEL ĐẸP ---
            
            # Lấy danh sách tất cả các tên cột của Hóa đơn (trừ cột details) để làm thông tin chung
            # Mục đích: Mỗi dòng sản phẩm sẽ kèm theo thông tin số hóa đơn, ngày tháng, tên người mua...
            meta_columns = [k for k in raw_data[0].keys() if k != 'details']

            # Dùng json_normalize để "bung" cột details ra
            df = pd.json_normalize(
                raw_data, 
                record_path=['details'],     # Cột chứa list sản phẩm con
                meta=meta_columns,           # Các cột thông tin cha cần giữ lại
                errors='ignore'
            )

            # Xem trước 5 dòng
            print(df.head())

            # --- LƯU RA EXCEL ---
            file_name = "DanhSachHoaDon_ChiTiet.csv"
            df.to_csv(file_name, index=False)
            
            print(f"\n✅ Đã lưu thành công file: {file_name}")
            print(f"Tổng số dòng dữ liệu: {len(df)}")
        else:
            print("Không có dữ liệu để lưu.")
            
    else:
        print("Lỗi API:", result.get("message"))

except Exception as e:
    print(f"Lỗi hệ thống: {e}")

Lỗi hệ thống: Conflicting metadata name inv_TotalAmount, need distinguishing prefix 
